In [1]:
path <- '/home/User1/data/self_citations/model/'

### Predict papers n years

In [2]:
year_start = 5
diff_year = 10
year_pred = year_start+diff_year

In [3]:
total_years_file <- 20
col_status <- rep("NULL",total_years_file+1)
col_status[c(1,year_start+1,year_start+diff_year+1)] <- "integer"
data <- read.csv(file=paste0(path,'nb_articles.csv'),colClasses =col_status)

In [ ]:
col_status <- rep("NULL",total_years_file+1)
col_status[c(1,year_start+1,year_start+diff_year+1)] <- "double"
types_cit <- c('self','co','others')
dfs_nb_cit <- list()
for(i in 1:length(types_cit)){
    type_cit <- types_cit[i]
    dfs_nb_cit[[i]] <- read.csv(file=paste0(path,'nb_citations_',type_cit,'.csv'),colClasses=col_status)
}

In [ ]:
authors_info <- read.csv(file=paste0(path,'authors_infos_model.csv'),sep='\t')

In [ ]:
for(i in 1:length(types_cit)){
    data <- merge(data,dfs_nb_cit[[i]],by="Cluster_ID")
}

In [ ]:
rm(dfs_nb_cit)

In [ ]:
data <- merge(data,authors_info,by="Cluster_ID")

In [ ]:
nb_coauthors <- read.csv(file=paste0(path,'nb_coauthors.csv'),colClasses =col_status)

In [ ]:
colnames(nb_coauthors)[c(2,3)] <- c(paste0("nb_co_",year_start),paste0("nb_co_",year_start+diff_year))

In [19]:
head(nb_coauthors)

,Cluster_ID,nb_co_5,nb_co_15
,<dbl>,<dbl>,<dbl>
1,31905805,5,5
2,10925507,7,7
3,37274983,9,33
4,11452931,13,13
5,38788724,5,5
6,43195937,3,3


In [ ]:
col_status_char <- rep("NULL",total_years_file+1)
col_status_char[c(1,year_start+1,year_start+diff_year+1)] <- "character"

In [ ]:
countries <- read.csv(file=paste0(path,'country_authors.csv'),colClasses =col_status_char)

In [ ]:
data <- merge(data,nb_coauthors,by="Cluster_ID")

In [ ]:
data <- merge(data,countries,by="Cluster_ID")

In [ ]:
data$discipline <- as.character(data$discipline)

In [ ]:
groups_disc <- read.csv(file=paste0(path,'groups_disc.csv'),header=F)

In [ ]:
colnames(groups_disc) <- c("group","discipline")

In [ ]:
data <- merge(data,groups_disc,by='discipline')

In [ ]:
# As time of start
min_year <- 1995
max_year <- 2007
# Min year car start
min_year_start  <- min_year - year_start
max_year_start <- max_year - year_start

data_model <- data[which((data['min_year'] >=min_year_start) & (data['min_year'] <= max_year_start)),]

In [ ]:
list_disciplines <- unique(data_model$discipline)

In [ ]:
groups <- unique(data_model$group)

In [86]:
head(data_model,5)

,Cluster_ID,nb_art_5,nb_art_15,nb_cits_self_5,nb_cits_self_15,nb_cits_co_5,nb_cits_co_15,nb_cits_others_5,nb_cits_others_15,discipline,min_year,max_year,nb_co_5,nb_co_15,country_5,country_15
,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>,<chr>,<chr>
24,53,7,124,0.00,265.03,0.0,79.40,5.64,1268.23,Clinical Medicine,1990,2018,13,241,Japan,Japan
25,54,5,28,9.34,59.39,0.0,27.27,22.70,456.12,Clinical Medicine,1992,2015,11,62,Japan,Japan
26,55,5,13,0.00,1.46,0.0,1.38,7.96,151.92,Clinical Medicine,1998,2012,22,38,Japan,Japan
28,57,48,84,256.10,423.26,352.4,1277.60,81.06,421.19,Physics,1999,2017,362,562,Japan,Japan
40,100,3,16,0.00,2.51,0.0,0.00,0.00,9.59,Clinical Medicine,2001,2018,9,25,Slovenia,Slovenia


#### Run model

install.packages("lme4")
install.packages("arm")
install.packages("lm.beta")

In [ ]:
library(lme4) 
library(arm)
library(lm.beta)

In [142]:
dep <- paste("nb_art",year_pred,sep="_")
var_base_dv_cits <- c("nb_cits_others","nb_co","nb_art","rank")
var_base <- c("nb_cits_self","nb_cits_co","nb_cits_others","nb_co")
ind_names <- c("nb_cits_others","nb_cits_others","nb_co")
ind_var <- lapply(ind_names,function(x) paste(x,year_start,sep="_"))
formula <- as.formula(paste0(dep,'~',paste(ind_var, collapse="+")))

#### conditions


In [143]:
country <- "United States"
disc <- list_disciplines[4]

In [144]:
for(group in groups){
    model <- lm(formula,data=data_model[(data_model[paste0('country_',year_start)]==country) &
                                   (data_model['group'] == group),])
}

[1] "Social Sciences"

In [146]:
display(model)

lm(formula = formula, data = data_model[(data_model[paste0("country_", 
    year_start)] == country) & (data_model["discipline"] == disc), 
    ])
                 coef.est coef.se
(Intercept)      16.52     0.19  
nb_cits_others_5  0.04     0.00  
nb_co_5           0.54     0.01  
---
n = 21144, k = 3
residual sd = 24.60, R-Squared = 0.16


In [125]:
model.beta <- lm.beta(model)

In [126]:
model.beta


Call:
lm(formula = formula, data = data_model[(data_model[paste0("country_", 
    year_start)] == country) & (data_model["discipline"] == disc), 
    ])

Standardized Coefficients::
     (Intercept) nb_cits_others_5          nb_co_5 
       0.0000000        0.1891615        0.4732701 
